### Calculate returns across all strike prices for all strategies
#### Option Returns Data Elements
- option_strategy
- entry_date
- moneyness_buy_to_open_call
- moneyness_sell_to_open_call
- moneyness_buy_to_open_put
- moneyness_sell_to_open_put
- pct_return
- market_direction
- sampling_key

Moneyness refers to the probability that the strike price that was bought or sold will close in the money on expiry date.
<br/>
 Each strategy will have its own output file and only the moneyness fields relevent to the strategy will be saved

#### The following csv files will be created
- buy_call_returns
- buy_put_returns
- buy_call_spread_returns
- buy_put_spread_returns
- sell_call_spread_returns
- sell_put_spread_returns



In [1]:
import pandas as pd
import csv
import datetime
from google.cloud import storage
import os

BUCKET_NAME = 'expiry-week-data'

In [2]:
#upload file to Google Cloud Storage 
def upload_blob(source_file_path):
    """Upload a file to Cloud Storage bucket."""
    
    filename = os.path.basename(source_file_path)
    destination_blob_name = "analytics/{}".format(filename)
 
    storage_client = storage.Client()
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_path)

    print(
        "File {} uploaded to {}.".format(
            source_file_path, destination_blob_name
        )
    )


#### Strategy: Buy Call

In [152]:
def buy_call_return(entry_call_ask, exit_call_price):
    """calculate percentage return"""
    if exit_call_price == 0:
        return -100.0
    pct_return = round((exit_call_price - entry_call_ask) / entry_call_ask * 100, 0)
    return pct_return


In [165]:
def buy_call():
    """
    Calculate returns for Buy Call Strategy
    - assign a moneyness bucket to each entry from .10 through to .90
    - for each moneyness bucket, keep only the strike price with the moneyness closest to the bucket value
    - we are dropping the .05 and .95 buckets as they could be sparsely populated 
    """
    spy_1wk_options = pd.read_csv('data/spy_1wk_options.csv', parse_dates=['entry_date', 'exit_date', 'expiry_date'])
    spy_1wk_options['moneyness_bucket'] = (spy_1wk_options['entry_call_moneyness'] * 1000 - spy_1wk_options['entry_call_moneyness'] * 1000 % 50) / 1000
    near_bucket_strikes = spy_1wk_options.groupby(['entry_date', 'moneyness_bucket'])['strike_price'].max().to_frame(name='strike_price').reset_index()
    sample_strikes = pd.merge(spy_1wk_options, near_bucket_strikes, how='inner', on=['entry_date', 'moneyness_bucket','strike_price'])
    sample_strikes['pct_return'] = sample_strikes.apply(lambda x: buy_call_return(x['entry_call_ask'], x['exit_call_price']), axis=1)

    return sample_strikes

    filepath = 'data/buy_call_returns.csv'
    
    with open(filepath, 'w', newline='') as f:
        out_csv = csv.writer(f)
        columns = ['entry_date', 'moneyness_bucket', 'pct_return', 'market_direction', 'sampling_key']
        
        out_csv.writerow(columns)
        
        for row in sample_strikes.itertuples():
            if row.moneyness_bucket >= .1 and row.moneyness_bucket <= .90:
                record = [
                    datetime.datetime.date(row.entry_date),
                    round(row.moneyness_bucket * 100, 0),
                    row.pct_return,
                    row.market_direction,
                    row.sampling_key]

                out_csv.writerow(record) 
            
        #upload file to Google Cloud Storage
        upload_blob(filepath)
    
#run function
#buy_call()
    

#### Strategy: Buy Put













In [157]:
def buy_put_return(entry_put_ask, exit_put_price):
    """calculate percentage return"""
    if exit_put_price == 0:
        return -100.0
    pct_return = round((exit_put_price - entry_put_ask) / entry_put_ask * 100, 0)
    return pct_return

In [7]:
def buy_put():
    """
    Calculate returns for Buy Put Strategy    
    """
    spy_1wk_options = pd.read_csv('data/spy_1wk_options.csv', parse_dates=['entry_date', 'exit_date', 'expiry_date'])
    spy_1wk_options['pct_return'] = spy_1wk_options.apply(lambda x: buy_put_return(x['entry_put_ask'], x['exit_put_price']), axis=1)
    filepath = 'data/buy_put_returns.csv'
    
    with open(filepath, 'w', newline='') as f:
        out_csv = csv.writer(f)
        columns = ['entry_date', 'moneyness_buy_to_open_put', 'pct_return', 'market_direction', 'sampling_key']
        
        out_csv.writerow(columns)
        
        for row in spy_1wk_options.itertuples():
            record = [
                datetime.datetime.date(row.entry_date),
                round(row.entry_put_moneyness, 3),
                row.pct_return,
                row.market_direction,
                row.sampling_key]
            out_csv.writerow(record)    
            
        #upload file to Google Cloud Storage
        upload_blob(filepath)
    
#run function
#buy_put()

File data/buy_put_returns.csv uploaded to analytics/buy_put_returns.csv.


#### Strategy: Buy Call Spread

In [158]:
def buy_call_spread_return(entry_buy_strike_ask, entry_sell_strike_bid, exit_buy_strike_price, exit_sell_strike_price):
    """calculate percentage return"""
    if exit_buy_strike_price == 0:
        return -100.0
    spread_cost = entry_buy_strike_ask - entry_sell_strike_bid
    spread_proceeds = exit_buy_strike_price - exit_sell_strike_price
    pct_return = round((spread_proceeds - spread_cost) / spread_cost * 100, 0)
    return pct_return

In [9]:
def buy_call_spread():
    """
    Calculate returns for Buy Call Spread Strategy
    """
    spy_1wk_options = pd.read_csv('data/spy_1wk_options.csv', parse_dates=['entry_date', 'exit_date', 'expiry_date'])
    entry_dates = spy_1wk_options['entry_date'].unique()
    filepath = 'data/buy_call_spread_returns.csv'
    
    with open(filepath, 'w', newline='') as f:
        out_csv = csv.writer(f)
        columns = ['entry_date', 'moneyness_buy_to_open_call', 'moneyness_sell_to_open_call',
            'pct_return', 'market_direction', 'sampling_key']
        
        out_csv.writerow(columns)
    
        for entry_date in entry_dates:
            option_set = spy_1wk_options[spy_1wk_options['entry_date'] == entry_date]
            row_count = option_set.shape[0]

            #pair each strike price record with every other strike price record
            for i in range(row_count):
                for j in range(i + 1, row_count):
                    buy_strike_record = option_set.iloc[i]
                    sell_strike_record = option_set.iloc[j]
                    
                    pct_return = buy_call_spread_return(
                            buy_strike_record['entry_call_ask'],
                            sell_strike_record['entry_call_bid'],
                            buy_strike_record['exit_call_price'],
                            sell_strike_record['exit_call_price'])

                    record = [
                        datetime.datetime.date(buy_strike_record['entry_date']),
                        round(buy_strike_record['entry_call_moneyness'], 3),
                        round(sell_strike_record['entry_call_moneyness'], 3),
                        pct_return,
                        buy_strike_record['market_direction'],
                        buy_strike_record['sampling_key']]
                    out_csv.writerow(record)   
                    
        #upload file to Google Cloud Storage
        upload_blob(filepath)

#run function
buy_call_spread()
                    

File data/buy_call_spread_returns.csv uploaded to analytics/buy_call_spread_returns.csv.


In [162]:
test = pd.read_csv('data/buy_call_returns.csv')
test.head()
max_id = test['pct_return'].idxmax()

In [176]:
sample_strikes = buy_call()
test2 = sample_strikes[sample_strikes['entry_date'] == '2017-12-29']
test2.iloc[12]

entry_date              2017-12-29 00:00:00
exit_date               2018-01-05 00:00:00
expiry_date             2018-01-05 00:00:00
strike_price                            269
entry_stock_price                    266.94
entry_atm_price                      266.27
entry_atm_iv                          0.061
entry_call_bid                         0.09
entry_call_ask                          0.1
entry_call_moneyness                  0.113
entry_put_bid                          2.68
entry_put_ask                          2.92
entry_put_moneyness                   0.887
exit_stock_price                      273.4
exit_call_price                         4.4
exit_put_price                            0
market_direction                  Strong Up
sampling_key                          0.442
moneyness_bucket                        0.1
pct_return                             4300
Name: 1929, dtype: object